In [1]:
#ppo algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NVDA_TRAINING.csv'
    test_file = 'NVDA_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the PPO model and train
    model = PPO("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1865 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1497        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006927577 |
|    clip_fraction        | 0.0344      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -2.71       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0168     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00649    |
|    value_loss         

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.93e+03     |
|    ep_rew_mean          | 0.86         |
| time/                   |              |
|    fps                  | 1339         |
|    iterations           | 12           |
|    time_elapsed         | 18           |
|    total_timesteps      | 24576        |
| train/                  |              |
|    approx_kl            | 0.0072234278 |
|    clip_fraction        | 0.0651       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.02        |
|    explained_variance   | 0.277        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.001        |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00967     |
|    value_loss           | 0.00133      |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+03    |
|    ep_rew_mean          | 4.01        |
| time/                   |             |
|    fps                  | 1331        |
|    iterations           | 22          |
|    time_elapsed         | 33          |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.011513409 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.958      |
|    explained_variance   | 0.143       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0128      |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.0145     |
|    value_loss           | 0.000954    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+03    |
|    ep_rew_mean          | 9.5         |
| time/                   |             |
|    fps                  | 1330        |
|    iterations           | 32          |
|    time_elapsed         | 49          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.014328163 |
|    clip_fraction        | 0.12        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.744      |
|    explained_variance   | 0.628       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00581    |
|    n_updates            | 310         |
|    policy_gradient_loss | -0.0032     |
|    value_loss           | 0.00723     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.93e+03     |
|    ep_rew_mean          | 12.4         |
| time/                   |              |
|    fps                  | 1329         |
|    iterations           | 42           |
|    time_elapsed         | 64           |
|    total_timesteps      | 86016        |
| train/                  |              |
|    approx_kl            | 0.0067829196 |
|    clip_fraction        | 0.0928       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.815       |
|    explained_variance   | 0.319        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0107      |
|    n_updates            | 410          |
|    policy_gradient_loss | -0.00718     |
|    value_loss           | 0.0014       |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent buys at 29.793, Current Balance: 10000, Holdings: 1 Long
Agent closes long at 29.938, profit: 0.14499999999999957, Current Balance: 10000.145, Holdings: 0
Agent buys at 29.945, Current Balance: 10000.145, Holdings: 1 Long
Agent closes long at 30.095800000000004, profit: 0.15080000000000382, Current Balance: 10000.2958, Holdings: 0
Agent sells (short) at 30.512, Current Balance: 10000.2958, Holdings: 1 Short
Agent closes short at 30.196, profit: 0.31599999999999895, Current Balance: 10000.6118, Holdings: 0
Agent buys at 29.964, Current Balance: 10000.6118, Holdings: 1 Long
Agent closes long at 30.134, profit: 0.1700000000000017, Current Balance: 10000.7818, Holdings: 0
Agent sells (short) at 30.075, Current Balance: 10000.7818, Holdings: 1 Short
Agent closes short at 28.627000000000002, profit: 1.4479999999999968, Current Balance: 10002.229800000001, Holdings: 0
Agent buys at 28.7, Curren

In [2]:
#dqn algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NVDA_TRAINING.csv'
    test_file = 'NVDA_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the DQN model and train
    model = DQN("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.93e+03 |
|    ep_rew_mean      | 0.392    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1288     |
|    time_elapsed     | 24       |
|    total_timesteps  | 31732    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00038  |
|    n_updates        | 7907     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.93e+03 |
|    ep_rew_mean      | -0.413   |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1275     |
|    time_elapsed     | 49       |
|    total_timesteps  | 63464    |
| train/              |        


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent sells (short) at 29.938, Current Balance: 10000, Holdings: 1 Short
Agent closes short at 29.945, profit: -0.0070000000000014495, Current Balance: 9999.993, Holdings: 0
Agent sells (short) at 29.882999999999996, Current Balance: 9999.993, Holdings: 1 Short
Agent closes short at 28.519, profit: 1.3639999999999972, Current Balance: 10001.357, Holdings: 0
Agent sells (short) at 28.525, Current Balance: 10001.357, Holdings: 1 Short
Agent closes short at 28.5055, profit: 0.019499999999997186, Current Balance: 10001.3765, Holdings: 0
Agent buys at 28.196, Current Balance: 10001.3765, Holdings: 1 Long
Agent closes long at 28.37, profit: 0.1739999999999995, Current Balance: 10001.550500000001, Holdings: 0
Agent sells (short) at 27.945, Current Balance: 10001.550500000001, Holdings: 1 Short
Agent closes short at 27.964, profit: -0.01899999999999835, Current Balance: 10001.531500000001, Holdings: 0

In [3]:
#a2c algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with action logging for debugging
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NVDA_TRAINING.csv'
    test_file = 'NVDA_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the A2C model and train with more timesteps
    model = A2C("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=500000)  # Increased timesteps

    # Testing on the training data with action logging
    obs, _ = env_train.reset()
    done = False
    print("\n--- Training Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_train.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the training session
    env_train.generate_report()
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Testing on the testing data with action logging
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    print("\n--- Testing Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_test.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the testing session
    env_test.generate_report()
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")



# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| time/                 |           |
|    fps                | 886       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -0.978    |
|    explained_variance | -7.35e+04 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.115    |
|    value_loss         | 0.0243    |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 935      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.491   |
|    explained_variance | -0.525   |
|    learning_rate      | 0.0007   |
|   

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.419    |
| time/                 |          |
|    fps                | 974      |
|    iterations         | 1700     |
|    time_elapsed       | 8        |
|    total_timesteps    | 8500     |
| train/                |          |
|    entropy_loss       | -0.602   |
|    explained_variance | -21.4    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1699     |
|    policy_loss        | -0.0111  |
|    value_loss         | 0.000998 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.419    |
| time/                 |          |
|    fps                | 975      |
|    iterations         | 1800     |
|    time_elapsed       | 9        |
|    total_timesteps    | 9000     |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.419    |
| time/                 |          |
|    fps                | 981      |
|    iterations         | 3000     |
|    time_elapsed       | 15       |
|    total_timesteps    | 15000    |
| train/                |          |
|    entropy_loss       | -0.45    |
|    explained_variance | -2.84    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2999     |
|    policy_loss        | -0.0011  |
|    value_loss         | 2.37e-06 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | 0.419     |
| time/                 |           |
|    fps                | 982       |
|    iterations         | 3100      |
|    time_elapsed       | 15        |
|    total_timesteps    | 15500     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.866    |
| time/                 |          |
|    fps                | 986      |
|    iterations         | 4300     |
|    time_elapsed       | 21       |
|    total_timesteps    | 21500    |
| train/                |          |
|    entropy_loss       | -0.373   |
|    explained_variance | -383     |
|    learning_rate      | 0.0007   |
|    n_updates          | 4299     |
|    policy_loss        | 0.000858 |
|    value_loss         | 8.51e-06 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | 0.866     |
| time/                 |           |
|    fps                | 987       |
|    iterations         | 4400      |
|    time_elapsed       | 22        |
|    total_timesteps    | 22000     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 1.05     |
| time/                 |          |
|    fps                | 987      |
|    iterations         | 5600     |
|    time_elapsed       | 28       |
|    total_timesteps    | 28000    |
| train/                |          |
|    entropy_loss       | -0.478   |
|    explained_variance | 0.538    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5599     |
|    policy_loss        | 0.00942  |
|    value_loss         | 5.86e-05 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | 1.05      |
| time/                 |           |
|    fps                | 987       |
|    iterations         | 5700      |
|    time_elapsed       | 28        |
|    total_timesteps    | 28500     |
| train/                |    

...

Step: 2336, Action: 0, Reward: 0
Step: 2337, Action: 0, Reward: 0
Step: 2338, Action: 0, Reward: 0
Step: 2339, Action: 0, Reward: 0
Step: 2340, Action: 0, Reward: 0
Step: 2341, Action: 0, Reward: 0
Step: 2342, Action: 0, Reward: 0
Step: 2343, Action: 0, Reward: 0
Step: 2344, Action: 0, Reward: 0
Step: 2345, Action: 0, Reward: 0
Step: 2346, Action: 0, Reward: 0
Step: 2347, Action: 0, Reward: 0
Step: 2348, Action: 0, Reward: 0
Step: 2349, Action: 0, Reward: 0
Step: 2350, Action: 0, Reward: 0
Step: 2351, Action: 0, Reward: 0
Step: 2352, Action: 0, Reward: 0
Step: 2353, Action: 0, Reward: 0
Step: 2354, Action: 0, Reward: 0
Step: 2355, Action: 0, Reward: 0
Step: 2356, Action: 0, Reward: 0
Step: 2357, Action: 0, Reward: 0
Step: 2358, Action: 0, Reward: 0
Step: 2359, Action: 0, Reward: 0
Step: 2360, Action: 0, Reward: 0
Step: 2361, Action: 0, Reward: 0
Step: 2362, Action: 0, Reward: 0
Step: 2363, Action: 0, Reward: 0
Step: 2364, Action: 0, Reward: 0
Step: 2365, Action: 0, Reward: 0
Step: 2366

Step: 2770, Action: 2, Reward: 0
Step: 2771, Action: 2, Reward: 0
Step: 2772, Action: 2, Reward: 0
Step: 2773, Action: 2, Reward: 0
Step: 2774, Action: 2, Reward: 0
Step: 2775, Action: 2, Reward: 0
Step: 2776, Action: 2, Reward: 0
Step: 2777, Action: 2, Reward: 0
Step: 2778, Action: 2, Reward: 0
Step: 2779, Action: 2, Reward: 0
Step: 2780, Action: 2, Reward: 0
Step: 2781, Action: 2, Reward: 0
Step: 2782, Action: 0, Reward: 0
Step: 2783, Action: 0, Reward: 0
Step: 2784, Action: 2, Reward: 0
Step: 2785, Action: 2, Reward: 0
Step: 2786, Action: 2, Reward: 0
Step: 2787, Action: 2, Reward: 0
Step: 2788, Action: 2, Reward: 0
Step: 2789, Action: 2, Reward: 0
Step: 2790, Action: 2, Reward: 0
Step: 2791, Action: 2, Reward: 0
Step: 2792, Action: 2, Reward: 0
Step: 2793, Action: 2, Reward: 0
Step: 2794, Action: 2, Reward: 0
Step: 2795, Action: 2, Reward: 0
Step: 2796, Action: 2, Reward: 0
Step: 2797, Action: 2, Reward: 0
Step: 2798, Action: 2, Reward: 0
Step: 2799, Action: 2, Reward: 0
Step: 2800

Step: 3206, Action: 2, Reward: 0
Step: 3207, Action: 2, Reward: 0
Step: 3208, Action: 2, Reward: 0
Step: 3209, Action: 2, Reward: 0
Step: 3210, Action: 2, Reward: 0
Step: 3211, Action: 2, Reward: 0
Step: 3212, Action: 2, Reward: 0
Step: 3213, Action: 2, Reward: 0
Step: 3214, Action: 2, Reward: 0
Step: 3215, Action: 2, Reward: 0
Step: 3216, Action: 2, Reward: 0
Step: 3217, Action: 2, Reward: 0
Step: 3218, Action: 2, Reward: 0
Step: 3219, Action: 2, Reward: 0
Step: 3220, Action: 2, Reward: 0
Step: 3221, Action: 2, Reward: 0
Step: 3222, Action: 2, Reward: 0
Step: 3223, Action: 2, Reward: 0
Step: 3224, Action: 2, Reward: 0
Step: 3225, Action: 2, Reward: 0
Step: 3226, Action: 2, Reward: 0
Step: 3227, Action: 2, Reward: 0
Step: 3228, Action: 2, Reward: 0
Step: 3229, Action: 2, Reward: 0
Step: 3230, Action: 2, Reward: 0
Step: 3231, Action: 2, Reward: 0
Step: 3232, Action: 2, Reward: 0
Step: 3233, Action: 2, Reward: 0
Step: 3234, Action: 2, Reward: 0
Step: 3235, Action: 2, Reward: 0
Step: 3236

Step: 3644, Action: 0, Reward: 0
Step: 3645, Action: 0, Reward: 0
Step: 3646, Action: 0, Reward: 0
Step: 3647, Action: 0, Reward: 0
Step: 3648, Action: 0, Reward: 0
Step: 3649, Action: 0, Reward: 0
Step: 3650, Action: 0, Reward: 0
Step: 3651, Action: 2, Reward: 0
Step: 3652, Action: 2, Reward: 0
Step: 3653, Action: 2, Reward: 0
Step: 3654, Action: 0, Reward: 0
Step: 3655, Action: 2, Reward: 0
Step: 3656, Action: 0, Reward: 0
Step: 3657, Action: 0, Reward: 0
Step: 3658, Action: 0, Reward: 0
Step: 3659, Action: 0, Reward: 0
Step: 3660, Action: 0, Reward: 0
Step: 3661, Action: 0, Reward: 0
Step: 3662, Action: 0, Reward: 0
Step: 3663, Action: 0, Reward: 0
Step: 3664, Action: 0, Reward: 0
Step: 3665, Action: 0, Reward: 0
Step: 3666, Action: 0, Reward: 0
Step: 3667, Action: 0, Reward: 0
Step: 3668, Action: 0, Reward: 0
Step: 3669, Action: 0, Reward: 0
Step: 3670, Action: 0, Reward: 0
Step: 3671, Action: 0, Reward: 0
Step: 3672, Action: 0, Reward: 0
Step: 3673, Action: 0, Reward: 0
Step: 3674

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [4]:
#ensemble learning single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO, DQN, A2C
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Ensemble model function
def ensemble_predict(actions):
    # Convert numpy arrays to integers for each action
    actions = [int(action) for action in actions]
    # Perform a majority vote among the actions (hold, buy, sell)
    action_counts = Counter(actions)
    return action_counts.most_common(1)[0][0]

# Train and evaluate the ensemble model
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'NVDA_TRAINING.csv'
    test_file = 'NVDA_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize each model and train them separately
    ppo_model = PPO("MlpPolicy", env_train, verbose=1)
    dqn_model = DQN("MlpPolicy", env_train, verbose=1)
    a2c_model = A2C("MlpPolicy", env_train, verbose=1)

    # Train each model
    ppo_model.learn(total_timesteps=50000)
    dqn_model.learn(total_timesteps=50000)
    a2c_model.learn(total_timesteps=50000)

    # Test the ensemble model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_train.step(final_action)

    # Calculate and display training metrics
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the ensemble model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_test.step(final_action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display testing metrics
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1811 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1559        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010276195 |
|    clip_fraction        | 0.041       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -34.7       |
|   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+03    |
|    ep_rew_mean          | -0.16       |
| time/                   |             |
|    fps                  | 1349        |
|    iterations           | 11          |
|    time_elapsed         | 16          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.008465815 |
|    clip_fraction        | 0.0745      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.06       |
|    explained_variance   | -0.0629     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0336     |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0106     |
|    value_loss           | 0.0012      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.93e+03    |
|    ep_rew_mean          | 2.7         |
| time/                   |             |
|    fps                  | 1330        |
|    iterations           | 21          |
|    time_elapsed         | 32          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.006683612 |
|    clip_fraction        | 0.0872      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.931      |
|    explained_variance   | 0.0774      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0353     |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.00763    |
|    value_loss           | 0.00424     |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.93

------------------------------------
| time/                 |          |
|    fps                | 979      |
|    iterations         | 800      |
|    time_elapsed       | 4        |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -0.772   |
|    explained_variance | -37.5    |
|    learning_rate      | 0.0007   |
|    n_updates          | 799      |
|    policy_loss        | -0.0116  |
|    value_loss         | 0.000552 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 979      |
|    iterations         | 900      |
|    time_elapsed       | 4        |
|    total_timesteps    | 4500     |
| train/                |          |
|    entropy_loss       | -0.867   |
|    explained_variance | -2.76    |
|    learning_rate      | 0.0007   |
|    n_updates          | 899      |
|    policy_loss        | 0.154    |
|    value_loss         | 0.0517   |
-

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | -0.169   |
| time/                 |          |
|    fps                | 983      |
|    iterations         | 2300     |
|    time_elapsed       | 11       |
|    total_timesteps    | 11500    |
| train/                |          |
|    entropy_loss       | -0.601   |
|    explained_variance | -149     |
|    learning_rate      | 0.0007   |
|    n_updates          | 2299     |
|    policy_loss        | -0.00385 |
|    value_loss         | 0.000331 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | -0.169   |
| time/                 |          |
|    fps                | 984      |
|    iterations         | 2400     |
|    time_elapsed       | 12       |
|    total_timesteps    | 12000    |
| train/                |          |
|

...

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | -0.0836   |
| time/                 |           |
|    fps                | 975       |
|    iterations         | 4900      |
|    time_elapsed       | 25        |
|    total_timesteps    | 24500     |
| train/                |           |
|    entropy_loss       | -0.258    |
|    explained_variance | -3.44     |
|    learning_rate      | 0.0007    |
|    n_updates          | 4899      |
|    policy_loss        | -1.27e-05 |
|    value_loss         | 1.18e-06  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | -0.0836  |
| time/                 |          |
|    fps                | 974      |
|    iterations         | 5000     |
|    time_elapsed       | 25       |
|    total_timesteps    | 25000    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | -0.0836  |
| time/                 |          |
|    fps                | 922      |
|    iterations         | 6200     |
|    time_elapsed       | 33       |
|    total_timesteps    | 31000    |
| train/                |          |
|    entropy_loss       | -0.0563  |
|    explained_variance | -731     |
|    learning_rate      | 0.0007   |
|    n_updates          | 6199     |
|    policy_loss        | 3.78e-05 |
|    value_loss         | 2.12e-05 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | -0.0836   |
| time/                 |           |
|    fps                | 922       |
|    iterations         | 6300      |
|    time_elapsed       | 34        |
|    total_timesteps    | 31500     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.401    |
| time/                 |          |
|    fps                | 926      |
|    iterations         | 7500     |
|    time_elapsed       | 40       |
|    total_timesteps    | 37500    |
| train/                |          |
|    entropy_loss       | -0.679   |
|    explained_variance | -0.601   |
|    learning_rate      | 0.0007   |
|    n_updates          | 7499     |
|    policy_loss        | -0.00474 |
|    value_loss         | 0.000313 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.93e+03  |
|    ep_rew_mean        | 0.401     |
| time/                 |           |
|    fps                | 927       |
|    iterations         | 7600      |
|    time_elapsed       | 40        |
|    total_timesteps    | 38000     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.655    |
| time/                 |          |
|    fps                | 935      |
|    iterations         | 8800     |
|    time_elapsed       | 47       |
|    total_timesteps    | 44000    |
| train/                |          |
|    entropy_loss       | -1.07    |
|    explained_variance | -9.38    |
|    learning_rate      | 0.0007   |
|    n_updates          | 8799     |
|    policy_loss        | -0.00475 |
|    value_loss         | 2.99e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 7.93e+03 |
|    ep_rew_mean        | 0.655    |
| time/                 |          |
|    fps                | 935      |
|    iterations         | 8900     |
|    time_elapsed       | 47       |
|    total_timesteps    | 44500    |
| train/                |          |
|


--- Training Metrics ---
Total Profit: 62.6709000000028
Cumulative Return: 0.0062670900000002805
Win Rate: 0.5440666204024983
Profit Factor: 1.8685629626344507
Sharpe Ratio: 0.14661302900559006
Sortino Ratio: 0.3112348782986499
Maximum Drawdown: 0.00038029986651420095

--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent buys at 29.793, Current Balance: 10000, Holdings: 1 Long
Agent closes long at 30.375999999999998, profit: 0.5829999999999984, Current Balance: 10000.583, Holdings: 0
Agent buys at 30.344, Current Balance: 10000.583, Holdings: 1 Long
Agent closes long at 29.72, profit: -0.6240000000000023, Current Balance: 9999.959, Holdings: 0
Agent buys at 28.627000000000002, Current Balance: 9999.959, Holdings: 1 Long
Agent closes long at 28.781, profit: 0.15399999999999636, Current Balance: 10000.113000000001, Holdings: 0
Agent buys at 28.5055, Current Balance: 10000.113000000001, Holdings: 1 Long
Agent closes long at 27.945, profit: -0